<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 4

## Using pandas for analysis

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

In [ ]:
import netCDF4

In [ ]:
snowcover_url = 'http://www.ncdc.noaa.gov/thredds/dodsC/cdr/snowcover/nhsce_v01r01_19661004_latest.nc'

open the opendap endpoint 

In [ ]:
%%time 
ds = netCDF4.Dataset(snowcover_url)

In [ ]:
ds.cdr_variable

In [ ]:
%%time
a = ds.variables['snow_cover_extent'][:, :, :]

In [ ]:
print(a.size, a.shape)

In [ ]:
plt.imshow(a[2000,:,:], cmap='Accent', interpolation='nearest')

In [ ]:
np.unique(a)

In [ ]:
time = ds.variables['time']

In [ ]:
times = netCDF4.num2date(time[:], time.units)

In [ ]:
data = ds.variables['snow_cover_extent'][0,:,:]

In [ ]:
data.shape()